In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
os.chdir("/Users/manhhung/Restaurant")
print("📂 Current working dir:", os.getcwd())



📂 Current working dir: /Users/manhhung/Restaurant


In [39]:
df = pd.read_csv("data/train_dataset/mi_quang.csv")
df.head(5)

,time_date,y_sales,dish_count_in_order_1d,dish_quantity_in_order_avg_1d,dish_base_price,dish_revenue_day_1d,dish_total_quantity_1d,day_order_count_1d,day_total_quantity_1d,day_total_revenue_1d,...,day_total_quantity_30d,day_total_revenue_30d,avg_order_size_30d,day_order_variety_30d,dish_count_in_order_30d,dish_quantity_in_order_avg_30d,dish_sales_ratio_daily_30d,dish_order_ratio_daily_30d,dish_revenue_ratio_daily_30d,dish_order_size_ratio_30d
0,2024-01-01,45,21,2.142857,45000,2025000,45,66,1579,72745000,...,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,2024-01-02,84,41,2.048780,45000,3780000,84,115,2575,123375000,...,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
2,2024-01-03,79,39,2.025641,45000,3555000,79,150,3045,150220000,...,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
3,2024-01-04,70,31,2.258065,45000,3150000,70,99,2066,96820000,...,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
4,2024-01-05,32,21,1.523810,45000,1440000,32,60,1168,55855000,...,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0


In [42]:
# ---- 1. Sắp xếp theo thời gian
df['time_date'] = pd.to_datetime(df['time_date'])
df = df.sort_values('time_date').reset_index(drop=True)

# ---- 2. Tạo feature từ time_date (tùy chọn, giúp mô hình hiểu mùa vụ)
df['day_of_week'] = df['time_date'].dt.weekday
df['month']       = df['time_date'].dt.month
df['day_of_year'] = df['time_date'].dt.dayofyear

# ---- 3. Chia theo tỷ lệ 60-20-20
n = len(df)
n_train = int(n * 0.6)
n_valid = int(n * 0.2)

train_df = df.iloc[:n_train]
valid_df = df.iloc[n_train:n_train+n_valid]
test_df  = df.iloc[n_train+n_valid:]

print(f"Train: {len(train_df)},  Valid: {len(valid_df)},  Test: {len(test_df)}")

# ---- 4. Chọn features và target
feature_cols = [c for c in df.columns if c not in ['y_sales','time_date']]
X_train, y_train = train_df[feature_cols], train_df['y_sales']
X_valid, y_valid = valid_df[feature_cols], valid_df['y_sales']
X_test , y_test  = test_df [feature_cols], test_df ['y_sales']

# ---- 5. Khởi tạo & train mô hình
model = XGBRegressor(
    n_estimators=800,
    learning_rate=0.025,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric='rmse',
    early_stopping_rounds=50,
    verbose=True
)

# ---- 6. Đánh giá trên tập test
pred = model.predict(X_test)

print("\n===== 📊 KẾT QUẢ TEST =====")
print("MAE :", mean_absolute_error(y_test, pred))
print("RMSE:", mean_squared_error(y_test, pred, squared=False))
print("R²  :", r2_score(y_test, pred))



Train: 219,  Valid: 73,  Test: 74
[0]	validation_0-rmse:51.52910
[1]	validation_0-rmse:50.46881
[2]	validation_0-rmse:49.36015
[3]	validation_0-rmse:48.27641
[4]	validation_0-rmse:47.21731
[5]	validation_0-rmse:46.18401
[6]	validation_0-rmse:45.16495
[7]	validation_0-rmse:44.19968
[8]	validation_0-rmse:43.23166
[9]	validation_0-rmse:42.26586
[10]	validation_0-rmse:41.40933
[11]	validation_0-rmse:40.55551
[12]	validation_0-rmse:39.68653
[13]	validation_0-rmse:38.82458
[14]	validation_0-rmse:38.21372
[15]	validation_0-rmse:37.43920
[16]	validation_0-rmse:36.66685
[17]	validation_0-rmse:35.92059
[18]	validation_0-rmse:35.19061
[19]	validation_0-rmse:34.46456
[20]	validation_0-rmse:33.75504
[21]	validation_0-rmse:33.06874
[22]	validation_0-rmse:32.37372
[23]	validation_0-rmse:31.69806
[24]	validation_0-rmse:31.06227
[25]	validation_0-rmse:30.44429
[26]	validation_0-rmse:29.80103
[27]	validation_0-rmse:29.18207
[28]	validation_0-rmse:28.59891
[29]	validation_0-rmse:28.05002
[30]	validation_

/opt/anaconda3/envs/py39ds/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/anaconda3/envs/py39ds/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[86]	validation_0-rmse:10.44157
[87]	validation_0-rmse:10.28879
[88]	validation_0-rmse:10.14718
[89]	validation_0-rmse:10.01300
[90]	validation_0-rmse:9.87089
[91]	validation_0-rmse:9.73341
[92]	validation_0-rmse:9.59512
[93]	validation_0-rmse:9.46144
[94]	validation_0-rmse:9.34105
[95]	validation_0-rmse:9.21304
[96]	validation_0-rmse:9.09302
[97]	validation_0-rmse:8.97527
[98]	validation_0-rmse:8.86188
[99]	validation_0-rmse:8.75360
[100]	validation_0-rmse:8.66274
[101]	validation_0-rmse:8.55464
[102]	validation_0-rmse:8.45033
[103]	validation_0-rmse:8.34458
[104]	validation_0-rmse:8.24096
[105]	validation_0-rmse:8.14303
[106]	validation_0-rmse:8.04352
[107]	validation_0-rmse:7.94748
[108]	validation_0-rmse:7.85708
[109]	validation_0-rmse:7.77451
[110]	validation_0-rmse:7.68575
[111]	validation_0-rmse:7.62468
[112]	validation_0-rmse:7.53985
[113]	validation_0-rmse:7.45692
[114]	validation_0-rmse:7.38555
[115]	validation_0-rmse:7.34402
[116]	validation_0-rmse:7.26568
[117]	validation_0

/opt/anaconda3/envs/py39ds/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
